# IDRNN
[Description]
(Add description here)

The goal of the project:
1. palceholder
2. placeholder


# Dataset

First download the dataset.

# Packages

In [1]:
!pip3 install -r requirements.txt


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
%load_ext autoreload
%autoreload 2
import h5py


import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec
import idx2numpy

import torch
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn as nn
import torch.nn.functional as F

import tqdm
from tqdm.notebook import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load data and print it

In [43]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Read the CSV file with sequences
data = pd.read_csv("data.csv", header=None)

# Define a function to convert amino acids to binary vectors
def amino_to_binary(amino):
    amino_to_index = {'A': 0, 'C': 1, 'D': 2, 'E': 3, 'F': 4, 'G': 5, 'H': 6, 'I': 7,
                      'K': 8, 'L': 9, 'M': 10, 'N': 11, 'P': 12, 'Q': 13, 'R': 14,
                      'S': 15, 'T': 16, 'V': 17, 'W': 18, 'Y': 19}
    
    index = amino_to_index.get(amino, -1)
    if index == -1:
        raise ValueError(f"Invalid amino acid: {amino}")
    
    binary_vector = [0] * 20
    binary_vector[index] = 1
    return binary_vector
  # Convert sequences to binary matrices
binary_sequence_matrices = []
max_sequence_length = 0

for index, row in data.iterrows():
    sequence = row[0]
    binary_sequence_matrix = [amino_to_binary(amino) for amino in sequence]
    binary_sequence_matrices.append(binary_sequence_matrix)
    
    if len(binary_sequence_matrix) > max_sequence_length:
        max_sequence_length = len(binary_sequence_matrix)

# Pad sequences to the same length
padded_binary_sequence_matrices = [seq + [[0] * 20] * (max_sequence_length - len(seq)) for seq in binary_sequence_matrices]

# Convert the padded matrices to a tensor
binary_sequence_tensor = torch.tensor(padded_binary_sequence_matrices, dtype=torch.float32)
# Split the data into a training set and a validation set
train_size = 0.8  # You can adjust this ratio
train_data, val_data = train_test_split(binary_sequence_tensor, train_size=train_size, random_state=42)

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Create dataset instances for training and validation
train_dataset = CustomDataset(train_data)
val_dataset = CustomDataset(val_data)

# Create data loaders for training and validation
batch_size = 64  # You can adjust this batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



In [44]:
# pick up one sequence to show how our data looks like
g = train_dataset[33]

print(g)
print(g.shape)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([981, 20])


# Helper functionform tuorial 6 and IUPred 

In [45]:
## Help function from tutroial 6 to calculate loss and plotting

def sample(mean, log_var):
    std = torch.exp(0.5*log_var)
    eps = torch.randn_like(std, device=mean.device)
    return mean + eps*std
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    return total_params
def plot_losses(reco_losses, kld_losses):
    fig = plt.figure(figsize=(10, 5))

    ax = fig.add_subplot(1, 2, 1)
    ax.plot(reco_losses)
    ax.set_title('Reconstruction Loss')

    ax = fig.add_subplot(1, 2, 2)
    ax.plot(kld_losses)
    ax.set_title('KLD Loss')
def generate_imgs(decoder_nn, cond=False):
    num_img = 32
    new_z = torch.randn(32, 2)
    if torch.cuda.is_available():
        new_z = new_z.cuda()

    if cond:
        new_labels = torch.zeros(num_img, 10).cuda()
        rand_int = torch.randint(0, 10, (num_img,)).cuda()
        new_labels[torch.arange(num_img), rand_int] = 1

        new_pred = decoder_nn(new_z, new_labels)
    else:
        new_pred = decoder_nn(new_z)

    new_pred = new_pred.cpu().detach().numpy()

    fig = plt.figure(figsize=(20, 10))
    gs = GridSpec(4, 8)

    for i in range(32):
        ax = fig.add_subplot(gs[i])
        img_array = new_pred[i][0]
        ax.imshow(img_array, cmap='gray')
        ax.axis('off')
        if cond:
            ax.set_title(f'Cond input: {rand_int[i]}')
def plot_z_dist(train_dl, encoder_nn, cond=False):
    fig = plt.figure(figsize=(24, 6))
    gs = GridSpec(1, 4)


    mus = []; log_vars = []; zs = []; ys = []
    for xb, yb, yb_one_hot in train_dl:
        if torch.cuda.is_available():
            xb = xb.cuda()
            yb_one_hot = yb_one_hot.cuda()

        if cond:
            mu, log_var = encoder_nn(xb, yb_one_hot)
        else:
            mu, log_var = encoder_nn(xb)
        z = sample(mu, log_var)

        mu = mu.cpu().detach().numpy()
        log_var = log_var.cpu().detach().numpy()
        z = z.cpu().detach().numpy()

        mus.extend(mu.tolist())
        log_vars.extend(log_var.tolist())
        zs.extend(z.tolist())
        ys.extend(yb.tolist())

    zs = np.array(zs)
    mus = np.array(mus)
    log_vars = np.array(log_vars)
    ys = np.array(ys)


    ax = fig.add_subplot(gs[0])
    ax.scatter(zs[:, 0], zs[:, 1], c=ys, cmap='tab10', alpha=0.3, s=1)
    ax.set_title('Z')
    ax.grid()
    ax.set_aspect('equal')

    gauss0 = np.random.normal(0, 1, len(zs[:, 0]))
    gauss1 = np.random.normal(0, 1, len(zs[:, 0]))
    ax = fig.add_subplot(gs[1])
    ax.scatter(gauss0, gauss1, c='cornflowerblue', alpha=0.3, s=1)
    ax.set_title('Gaus(0,1))')
    ax.set_aspect('equal')

    return ys, zs

# trainning loop based on tutorial 6

In [30]:
# Define the layer dimensions for encoder and decoder
encoder_layers = 981*20  # Input dimension, hidden layer size, and latent dimension
decoder_layers = 64    # Latent dimension, class dimension, and output dimension

# Define other parameters
hid_layer = 5  # Number of hidden layers
last_layer_activation = False  # Set to True if you want the last layer to have an activation function
latent_dim = 64

In [31]:
from model_VAE import EncoderLn, DecoderLn
encoder_nn = EncoderLn(encoder_layers, hid_layer, latent_dim, last_layer_activation)
decoder_nn = DecoderLn(decoder_layers, hid_layer, output_dim)

if torch.cuda.is_available():
    encoder_nn.cuda()
    decoder_nn.cuda()

TypeError: 'int' object is not subscriptable

# now i try to use NN form tutorial to see the mistake 

In [46]:
# Encoder
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.seq1 = nn.Sequential(
            nn.Linear(981*20, 300*20),
            nn.LeakyReLU(),

            nn.Linear(300*20, 200),
            nn.LeakyReLU(),

            nn.Linear(200, 100),
            nn.LeakyReLU()
        )

        self.mean_nn    = nn.Linear(100, 2)
        self.log_var_nn = nn.Linear(100, 2)


    def forward(self, x):
        x = x.view(-1, 981*20)
        x = self.seq1(x)

        mean = self.mean_nn(x)
        log_var = self.log_var_nn(x)

        return mean, log_var
    

# Decoder
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.seq1 = nn.Sequential(
            nn.Linear(2, 100),
            nn.LeakyReLU(),

            nn.Linear(100, 200),
            nn.LeakyReLU(),

            nn.Linear(200, 300*20),
            nn.LeakyReLU(),

            nn.Linear(300*20, 981*20),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.seq1(x)
        x = x.view(-1, 1, 981, 20)        

        return x
encoder_nn = Encoder()
decoder_nn = Decoder()

In [50]:

optimizer = torch.optim.Adam(list(encoder_nn.parameters()) + list(decoder_nn.parameters()), lr=0.001)
reco_losses = []
kld_losses  = []

for ep in tqdm(range(50)):
    for xb, yb in train_dataset:
        if torch.cuda.is_available():
            xb = xb.cuda()
            yb = yb.cuda()


        optimizer.zero_grad()

        mu, log_var = encoder_nn(xb)
        z = sample(mu, log_var)

        pred = decoder_nn(z)

        # loss
        reco_loss = BCE = F.binary_cross_entropy(pred, xb, reduction='sum')
        kld_loss  = -0.5 * torch.sum(1 + log_var - mu**2 -  log_var.exp())

        loss = reco_loss + kld_loss

        loss.backward()
        optimizer.step()

        reco_losses.append(reco_loss.item())
        kld_losses.append(kld_loss.item())



  0%|          | 0/50 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 2)